In [213]:
import nltk
import pandas as pd
import io
import requests

In [214]:
AND = 1
OR = 2

In [215]:
# Download o dado CSV
# Dependendo de sua internet, este comando pode demorar
def download_csv():
    r = requests.get('https://drive.google.com/a/ccc.ufcg.edu.br/uc?authuser=0&id=1YqUakSw2BLj-Hgz_oPntvHWwK6vKxjOb&output=csv')
    data = r.content
    return data

In [216]:
#Formata o CSV como um Data Frame 
# Contéudo do DF:
# titulo: Titulo da Noticia
# conteudo: Contéudo da Noticia
# idNoticia: Id da noticia
# content: O titulo + ' ' conteudo (Concatenção do titulo e do conteudo separado por um ' ') 
def getCSVFormattedAsDF():
    data = download_csv()
    df = pd.read_csv(io.BytesIO(data))
    df["content"] = df["titulo"] + ' ' + df["conteudo"]
    return df

In [217]:
#Cria e retorna um dicionario de tokens. Cada chave do dicionario possui como valor um conjunto(set) de noticias em que 
# este token aparece. 
def init_tokens():
    tokens = {}
    df = getCSVFormattedAsDF()
    sizeDF = len(df["idNoticia"])
    
    for idx in range(sizeDF):
        content = df["content"][idx]
        words = nltk.word_tokenize(content)
        for word in words:
            word = word.lower()
            try:
                tokens[word].add(df["idNoticia"][idx]) 
            except KeyError:
                tokens[word] = set()
                tokens[word].add(df["idNoticia"][idx]) 
            
    return tokens

In [218]:
# Retorna o conjunto de noticias onde o tokens aparece. Caso o tokens não exista nos dados, retorna um set vazio. 
def getTokenInfo(key, tokens):
    if (key not in tokens):
        return set()
    else:
        return tokens[key]

In [219]:
# Retorna o conjunto de tokens da operação pesquisa. 
def search(ipt, tokens):
    if (ipt.strip() == ""):
        return "Please, inform at least one token"
    
    ipt = ipt.split(" ")
    init = ipt[0].lower()
    size = len(ipt)
    
    if (size == 1) :
        if init not in tokens:
            return "This token is not present in any news"
        else:
            return tokens[init]
    else:
        result = set(getTokenInfo(init,tokens))
        command = ""
        for index in range(1,size):
            if (index % 2 == 0):
                compare = getTokenInfo(ipt[index].lower(), tokens)
    
                if  command == AND:
                    result = result & compare
                else:
                    result = result | compare              
            else:
                if (ipt[index] != "AND" and ipt[index] != "OR"):
                    return "Incorrect input. The tokens searched only can be separated by AND-OR"
                elif ipt[index] == "AND":
                    command = AND
                else:
                    command = OR
    
        return result
        

In [220]:
# Inicia o dicionario de tokens
tokens = init_tokens()

In [221]:
assert len(search("Campina AND Grande", tokens)) == 12
assert len(search("debate OR presidencial", tokens)) == 1770
assert len(search("debate AND presidencial", tokens)) == 201
assert len(search("presidenciáveis OR corruptos", tokens)) == 164
assert len(search("presidenciáveis AND corruptos", tokens)) == 0
assert len(search("Belo OR Horizonte", tokens)) == 331
assert len(search("Belo AND Horizonte", tokens)) == 242

{1036, 1068, 1370, 1770, 1952, 1987, 2763, 2777, 2779, 4802, 5382, 5870, 6694}